In [ ]:
# Завантаження бібліотек
import numpy as np
from keras.datasets import cifar10
from keras.utils import to_categorical
import cv2
import time
from numba import njit # numba для прискорення математичних операцій

In [ ]:
import warnings
# Відключення попереджень про застарілість
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
np.random.seed(777)

In [ ]:
# Функція активації ReLU
@njit(fastmath=True)
def relu(x):
    return np.maximum(0, x)

In [ ]:
# Функція активації Softmax
@njit(fastmath=True)
def softmax(raw_preds):
    out = np.exp(raw_preds)
    return out/np.sum(out)

In [ ]:
#original
def convolution_layer(input_value, kernel, bias, stride=1, padding=0):
    padded_image = np.pad(input_value, ((0, 0), (padding, padding), (padding, padding)), mode='constant')
    (n_f, n_c_f, f, _) = kernel.shape
    n_c, in_dim, _ = padded_image.shape

    out_dim = int((in_dim - f)/stride)+1

    out = np.zeros((n_f,out_dim,out_dim))

    for curr_f in range(n_f):
        curr_y = out_y = 0
        while curr_y + f <= in_dim:
            curr_x = out_x = 0
            while curr_x + f <= in_dim:
                out[curr_f, out_y, out_x] = np.sum(kernel[curr_f] * padded_image[:,curr_y:curr_y+f, curr_x:curr_x+f]) + bias[curr_f]
                curr_x += stride
                out_x += 1
            curr_y += stride
            out_y += 1

    return out

In [ ]:
# Шар згортки
def conv_layer(input_tensor, filter_weights, filter_bias, step=1, pad=0):
    """
    Застосовує шар згортки до вхідного тензора

    Параметри:
    input_tensor - Вхідний тензор.
    filter_weights - Ваги фільтрів.
    filter_bias - Зміщення фільтрів.
    step - Крок згортки. За замовчуванням 1.
    pad - Розмір краю для padding. За замовчуванням 0.

    Повертає:
    np.array - Результат застосування шару згортки до вхідного тензора.
    """
    # padding вхідного тензора
    padded_tensor = np.pad(input_tensor, ((0, 0), (pad, pad), (pad, pad)), mode='constant')
    # Отримання розмірностей
    (num_filters, num_channels_filt, filt_size, _) = filter_weights.shape
    num_channels, input_size, _ = padded_tensor.shape
    # Розрахунок розмірності вихідного тензора
    output_size = int((input_size - filt_size) / step) + 1
    # Ініціалізація вихідного тензора
    output_tensor = np.zeros((num_filters, output_size, output_size))

    # Проходження фільтрами та розрахунок вихідних значень
    for filter_index in range(num_filters):
        input_y = output_y = 0
        while input_y + filt_size <= input_size:
            input_x = output_x = 0
            while input_x + filt_size <= input_size:
                output_tensor[filter_index, output_y, output_x] = np.sum(filter_weights[filter_index] * padded_tensor[:, input_y:input_y+filt_size, input_x:input_x+filt_size]) + filter_bias[filter_index]
                input_x += step
                output_x += 1
            input_y += step
            output_y += 1

    return output_tensor

In [ ]:
# Шар максимального пулінгу
def max_pooling(input_value, pool_size=2, stride=2):
    # Визначення розмірів вхідного значення
    input_depth, input_height, input_width = input_value.shape
    # Розрахунок висоти та ширини вихідного значення
    output_height = (input_height - pool_size) // stride + 1
    output_width = (input_width - pool_size) // stride + 1
    # Ініціалізація вихідного значення нулями
    output_value = np.zeros((input_depth, output_height, output_width))

    # Перебір по всіх глибинах вхідного значення
    for d in range(input_depth):
        # Перебір по висоті вхідного значення з урахуванням кроку
        for i in range(0, input_height - pool_size + 1, stride):
            # Перебір по ширині вхідного значення з урахуванням кроку
            for j in range(0, input_width - pool_size + 1, stride):
                # Обчислення максимального значення в поточному підвікні
                output_value[d, i // stride, j // stride] = np.max(
                    input_value[d, i:i + pool_size, j:j + pool_size])
    # Повернення вихідного значення
    return output_value

In [ ]:
# Функція втрат - категорійна крос-ентропія
@njit(fastmath=True)
def cross_entropy_loss(predicted, target):
    return -np.sum(target * np.log(predicted))

In [ ]:
# original
@njit(fastmath=True)
def convolutionBackward(dconv_prev, conv_in, filt, s):
    (n_f, n_c, f, _) = filt.shape
    (_, orig_dim, _) = conv_in.shape

    dout = np.zeros(conv_in.shape)
    dfilt = np.zeros(filt.shape)
    dbias = np.zeros((n_f,1))
    for curr_f in range(n_f):

        curr_y = out_y = 0
        while curr_y + f <= orig_dim:
            curr_x = out_x = 0
            while curr_x + f <= orig_dim:

                dfilt[curr_f] += dconv_prev[curr_f, out_y, out_x] * conv_in[:, curr_y:curr_y+f, curr_x:curr_x+f]

                dout[:, curr_y:curr_y+f, curr_x:curr_x+f] += dconv_prev[curr_f, out_y, out_x] * filt[curr_f]
                curr_x += s
                out_x += 1
            curr_y += s
            out_y += 1

        dbias[curr_f] = np.sum(dconv_prev[curr_f])

    return dout, dfilt, dbias

In [ ]:
# Зворотнє розповсюдження помилки для шару згортки
@njit(fastmath=True)
def conv_backward(d_out, input_tensor, filters, stride):
    """
    Обчислює зворотнє поширення для шару згортки.

    Параметри:
    d_out - Градієнт вихідного тензора.
    input_tensor - Вхідний тензор.
    filters - Ваги фільтрів.
    stride - Крок згортки.

    Повертає:
    d_input - Градієнт вхідного тензора.
    d_filters - Градієнт ваг фільтрів.
    d_biases - Градієнт зміщень фільтрів.
    """
    (num_filters, num_channels, filter_size, _) = filters.shape
    (_, input_dim, _) = input_tensor.shape

    d_input = np.zeros(input_tensor.shape)  # Ініціалізація градієнта вхідного тензора
    d_filters = np.zeros(filters.shape)  # Ініціалізація градієнта ваг фільтрів
    d_biases = np.zeros((num_filters, 1))  # Ініціалізація градієнта зміщень фільтрів

    for filter_index in range(num_filters):
        input_y = output_y = 0
        while input_y + filter_size <= input_dim:
            input_x = output_x = 0
            while input_x + filter_size <= input_dim:
                # Обчислення градієнту ваг фільтрів
                d_filters[filter_index] += d_out[filter_index, output_y, output_x] * input_tensor[:, input_y:input_y+filter_size, input_x:input_x+filter_size]
                # Обчислення градієнту вхідного тензора
                d_input[:, input_y:input_y+filter_size, input_x:input_x+filter_size] += d_out[filter_index, output_y, output_x] * filters[filter_index]
                input_x += stride
                output_x += 1
            input_y += stride
            output_y += 1
        # Обчислення градієнту зміщень фільтрів
        d_biases[filter_index] = np.sum(d_out[filter_index])

    return d_input, d_filters, d_biases

In [ ]:
#original
def nanArgMax(arr):
    idx = np.nanargmax(arr)
    idxs = np.unravel_index(idx, arr.shape)
    return idxs

def maxpoolBackward(dpool, orig, f, s):
    (n_c, orig_dim, _) = orig.shape

    dout = np.zeros(orig.shape)

    for curr_c in range(n_c):
        curr_y = 0
        out_y = 0
        while curr_y + f <= orig_dim:
            curr_x = 0
            out_x = 0
            while curr_x + f <= orig_dim:
                (a, b) = nanArgMax(orig[curr_c, curr_y:curr_y+f, curr_x:curr_x+f])
                dout[curr_c, curr_y+a, curr_x+b] += dpool[curr_c, out_y, out_x]

                curr_x += s
                out_x += 1
            curr_y += s
            out_y += 1

    return dout

In [ ]:
def nan_arg_max(array):
    # Знаходить позицію максимального значення у масиві, ігноруючи значення NaN.
    index = np.nanargmax(array)  # Знаходить індекс максимального значення у масиві
    indices = np.unravel_index(index, array.shape)  # Перетворює лінійний індекс у вимірні індекси
    return indices  # Повертає позицію максимального значення у масиві

def maxpool_backward(d_out, input_tensor, filter_size, stride):
    # Обчислює зворотнє поширення для шару максимального пулінгу.
    (num_channels, input_dim, _) = input_tensor.shape
    d_input = np.zeros(input_tensor.shape)  # Ініціалізує градієнт вхідного тензора

    for channel_index in range(num_channels):
        input_y = output_y = 0
        while input_y + filter_size <= input_dim:
            input_x = output_x = 0
            while input_x + filter_size <= input_dim:
                # Знаходить позицію максимального значення у підмасиві
                (max_y, max_x) = nan_arg_max(input_tensor[channel_index, input_y:input_y+filter_size, input_x:input_x+filter_size])
                # Додає градієнт до відповідного вхідного пікселя
                d_input[channel_index, input_y + max_y, input_x + max_x] += d_out[channel_index, output_y, output_x]
                input_x += stride
                output_x += 1
            input_y += stride
            output_y += 1
    return d_input  # Повертає градієнт вхідного тензора

In [ ]:
@njit(fastmath=True)
def update_func(velocity, squared_gradient, beta1, beta2, batch_size, gradient, parameter, learning_rate):
    """
    Оновлює параметр моделі за допомогою методу Adam.

    Параметри:
    velocity - Експоненційно зважене середнє градієнтів за минулі кроки.
    squared_gradient - Експоненційно зважене середнє квадратів градієнтів за минулі кроки.
    beta1 - Параметр згладжування для velocity.
    beta2 - Параметр згладжування для squared_gradient.
    batch_size - Розмір пакета даних.
    gradient - Градієнт.
    parameter - Параметр, який потрібно оновити.
    learning_rate - Швидкість навчання.

    Повертає:
    np.array - Оновлений параметр моделі.
    """

    velocity = beta1 * velocity + (1 - beta1) * gradient / batch_size  # Оновлюємо velocity за допомогою експоненційно зваженого середнього градієнтів
    squared_gradient = beta2 * squared_gradient + (1 - beta2) * (gradient / batch_size) ** 2  # Оновлюємо squared_gradient за допомогою експоненційно зваженого середнього квадратів градієнтів
    parameter -= learning_rate * velocity / np.sqrt(squared_gradient + 1e-7)  # Оновлюємо параметр parameter згідно формули Adam
    return parameter  # Повертаємо оновлений параметр parameter

In [ ]:
def adam(params, grads, learning_rate, batch_size, beta1, beta2):
    """
    Оновлює параметри за допомогою алгоритму Adam.

    Параметри:
    params - Список параметрів моделі.
    grads - Список градієнтів моделі.
    learning_rate - Швидкість навчання.
    batch_size - Розмір пакета даних.
    beta1 - Параметр згладжування для оновлення швидкості.
    beta2 - Параметр згладжування для оновлення квадрату градієнту.

    Повертає:
    list - Оновлені параметри моделі.
    """
    # Розпаковуємо параметри
    kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6, weights_f7, bias_f7, weights_output, bias_output = params

    # Ініціалізуємо змінні для velocity
    v1  = np.zeros(kernel_conv1.shape)
    bv1 = np.zeros(bias_conv1.shape)
    v2  = np.zeros(kernel_conv2.shape)
    bv2 = np.zeros(bias_conv2.shape)
    v3  = np.zeros(kernel_conv3.shape)
    bv3 = np.zeros(bias_conv3.shape)
    v4  = np.zeros(kernel_conv4.shape)
    bv4 = np.zeros(bias_conv4.shape)
    v5  = np.zeros(kernel_conv5.shape)
    bv5 = np.zeros(bias_conv5.shape)
    v6  = np.zeros(weights_f6.shape)
    bv6 = np.zeros(bias_f6.shape)
    v7  = np.zeros(weights_f7.shape)
    bv7 = np.zeros(bias_f7.shape)
    v8  = np.zeros(weights_output.shape)
    bv8 = np.zeros(bias_output.shape)

    # Ініціалізуємо змінні для squared_gradient
    s1  = np.zeros(kernel_conv1.shape)
    bs1 = np.zeros(bias_conv1.shape)
    s2  = np.zeros(kernel_conv2.shape)
    bs2 = np.zeros(bias_conv2.shape)
    s3  = np.zeros(kernel_conv3.shape)
    bs3 = np.zeros(bias_conv3.shape)
    s4  = np.zeros(kernel_conv4.shape)
    bs4 = np.zeros(bias_conv4.shape)
    s5  = np.zeros(kernel_conv5.shape)
    bs5 = np.zeros(bias_conv5.shape)
    s6  = np.zeros(weights_f6.shape)
    bs6 = np.zeros(bias_f6.shape)
    s7  = np.zeros(weights_f7.shape)
    bs7 = np.zeros(bias_f7.shape)
    s8  = np.zeros(weights_output.shape)
    bs8 = np.zeros(bias_output.shape)

    # Розпаковуємо градієнти
    d_kernel_conv1, d_bias_conv1, d_kernel_conv2, d_bias_conv2, d_kernel_conv3, d_bias_conv3, d_kernel_conv4, d_bias_conv4, d_kernel_conv5, d_bias_conv5, d_weights_f6, d_bias_f6, d_weights_f7, d_bias_f7, d_weights_output, d_bias_output = grads

    # Оновлюємо кожен параметр за допомогою методу Adam
    kernel_conv1 = update_func(v1, s1, beta1, beta2, batch_size, d_kernel_conv1, kernel_conv1, learning_rate)
    bias_conv1 = update_func(bv1, bs1, beta1, beta2, batch_size, d_bias_conv1, bias_conv1, learning_rate)
    kernel_conv2 = update_func(v2, s2, beta1, beta2, batch_size, d_kernel_conv2, kernel_conv2, learning_rate)
    bias_conv2 = update_func(bv2, bs2, beta1, beta2, batch_size, d_bias_conv2, bias_conv2, learning_rate)
    kernel_conv3 = update_func(v3, s3, beta1, beta2, batch_size, d_kernel_conv3, kernel_conv3, learning_rate)
    bias_conv3 = update_func(bv3, bs3, beta1, beta2, batch_size, d_bias_conv3, bias_conv3, learning_rate)
    kernel_conv4 = update_func(v4, s4, beta1, beta2, batch_size, d_kernel_conv4, kernel_conv4, learning_rate)
    bias_conv4 = update_func(bv4, bs4, beta1, beta2, batch_size, d_bias_conv4, bias_conv4, learning_rate)
    kernel_conv5 = update_func(v5, s5, beta1, beta2, batch_size, d_kernel_conv5, kernel_conv5, learning_rate)
    bias_conv5 = update_func(bv5, bs5, beta1, beta2, batch_size, d_bias_conv5, bias_conv5, learning_rate)
    weights_f6 = update_func(v6, s6, beta1, beta2, batch_size, d_weights_f6, weights_f6, learning_rate)
    bias_f6 = update_func(bv6, bs6, beta1, beta2, batch_size, d_bias_f6, bias_f6, learning_rate)
    weights_f7 = update_func(v7, s7, beta1, beta2, batch_size, d_weights_f7, weights_f7, learning_rate)
    bias_f7 = update_func(bv7, bs7, beta1, beta2, batch_size, d_bias_f7, bias_f7, learning_rate)
    weights_output = update_func(v8, s8, beta1, beta2, batch_size, d_weights_output, weights_output, learning_rate)
    bias_output = update_func(bv8, bs8, beta1, beta2, batch_size, d_bias_output, bias_output, learning_rate)

    # Групуємо оновлені параметри
    params = [kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6, weights_f7, bias_f7, weights_output, bias_output]

    # Повертаємо оновлені параметри
    return params

In [ ]:
def my_model():
    # Функція для прямого проходження мережі
    def forward_propagation(image, params):
    # Розпаковка параметрів для кожного шару мережі
        kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6, weights_f7, bias_f7, weights_output, bias_output = params

        # Прямий прохід через перші п'ять шарів: згорткові та пулінгові шари
        conv1_output = relu(convolution_layer(image, kernel_conv1, bias_conv1, stride=4))  # Застосування згорткового шару з ReLU активацією та зсувом
        pool1_output = max_pooling(conv1_output)  # Пулінговий шар
        conv2_output = relu(convolution_layer(pool1_output, kernel_conv2, bias_conv2, padding=2))  # Застосування згорткового шару з ReLU активацією та зсувом
        pool2_output = max_pooling(conv2_output)  # Пулінговий шар
        conv3_output = relu(convolution_layer(pool2_output, kernel_conv3, bias_conv3, padding=1))  # Застосування згорткового шару з ReLU активацією та зсувом
        conv4_output = relu(convolution_layer(conv3_output, kernel_conv4, bias_conv4, padding=1))  # Застосування згорткового шару з ReLU активацією та зсувом
        conv5_output = relu(convolution_layer(conv4_output, kernel_conv5, bias_conv5, padding=1))  # Застосування згорткового шару з ReLU активацією та зсувом
        pool5_output = max_pooling(conv5_output)  # Пулінговий шар

        # Підготовка вихідного вектора для повнозв'язаного шару
        (nf2, dim2, _) = pool5_output.shape
        fc = pool5_output.reshape((nf2 * dim2 * dim2, 1))

        # Прямий прохід через повнозв'язаний шар з ReLU активацією
        fc6_output = relu(weights_f6.dot(fc) + bias_f6)
        fc7_output = relu(weights_f7.dot(fc6_output) + bias_f7)
        output_value = softmax(weights_output.dot(fc7_output) + bias_output)

        # Пакування результатів для подальшого використання в зворотному проходженні
        params = [conv1_output, pool1_output, conv2_output, pool2_output, conv3_output, conv4_output, conv5_output, pool5_output, fc, fc6_output, fc7_output, output_value]

        return params

    # Функція для зворотного проходження мережі
    def backward_propagation(image, target, learning_rate, params):
        # Розпаковка параметрів
        kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6, weights_f7, bias_f7, weights_output, bias_output = params
        # Прямий прохід для отримання необхідних результатів
        conv1_output, pool1_output, conv2_output, pool2_output, conv3_output, conv4_output, conv5_output, pool5_output, fc, fc6_output, fc7_output, output_value = forward_propagation(image, params)
        # Розрахунок втрат
        loss = cross_entropy_loss(output_value, target)

        # Обчислення градієнтів відносно ваг та зсувів кожного шару
        d_out = output_value - target

        # Градієнти для вихідного шару
        d_weights_output = d_out.dot(fc7_output.T)
        d_bias_output = np.sum(d_out, axis=1).reshape(bias_output.shape)

        # Градієнти для повнозв'язаного шару 7
        d_fc7_output = weights_output.T.dot(d_out)
        d_fc7_output[fc7_output <= 0] = 0
        d_weights_f7 = d_fc7_output.dot(fc6_output.T)
        d_bias_f7 = np.sum(d_fc7_output, axis=1).reshape(bias_f7.shape)

        # Градієнти для повнозв'язаного шару 6
        d_fc6_output = weights_f7.T.dot(d_fc7_output)
        d_fc6_output[fc6_output <= 0] = 0
        d_weights_f6 = d_fc6_output.dot(fc.T)
        d_bias_f6 = np.sum(d_fc6_output, axis=1).reshape(bias_f6.shape)

        # Градієнти для згорткових шарів та їх параметрів
        d_fc = weights_f6.T.dot(d_fc6_output)
        d_pool5_output = d_fc.reshape(pool5_output.shape)

        d_conv5_output = maxpoolBackward(d_pool5_output, conv5_output, f=3, s=2)
        d_conv5_output[conv5_output <= 0] = 0

        d_conv4_output, d_kernel_conv5, d_bias_conv5 = convolutionBackward(d_conv5_output, conv4_output, kernel_conv5, s=1)
        d_conv4_output[conv4_output <= 0] = 0

        d_conv3_output, d_kernel_conv4, d_bias_conv4 = convolutionBackward(d_conv4_output, conv3_output, kernel_conv4, s=1)
        d_conv3_output[conv3_output <= 0] = 0

        d_pool2_output, d_kernel_conv3, d_bias_conv3 = convolutionBackward(d_conv3_output, pool2_output, kernel_conv3, s=1)
        d_pool2_output[pool2_output <= 0] = 0

        d_conv2_output = maxpoolBackward(d_pool2_output, conv2_output, f=3, s=2)
        d_conv2_output[conv2_output <= 0] = 0

        d_pool1_output, d_kernel_conv2, d_bias_conv2 = convolutionBackward(d_conv2_output, pool1_output, kernel_conv2, s=1)
        d_pool1_output[pool1_output <= 0] = 0

        d_conv1_output = maxpoolBackward(d_pool1_output, conv1_output, f=3, s=2)
        d_conv1_output[conv1_output <= 0] = 0

        d_image, d_kernel_conv1, d_bias_conv1 = convolutionBackward(d_conv1_output, image, kernel_conv1, s=4)

        # Підготовка градієнтів для подальшого використання
        grads = [d_kernel_conv1, d_bias_conv1, d_kernel_conv2, d_bias_conv2, d_kernel_conv3, d_bias_conv3, d_kernel_conv4, d_bias_conv4, d_kernel_conv5, d_bias_conv5, d_weights_f6, d_bias_f6, d_weights_f7, d_bias_f7, d_weights_output, d_bias_output]

        return grads

    return forward_propagation, backward_propagation

In [ ]:
def initialize_weights(shape):
    # Обчислення стандартного відхилення за методом Ксав'є
    stddev = np.sqrt(2.0 / np.prod(shape[:-1]))
    # Ініціалізація ваг за допомогою випадкового розподілу з нормальним розподілом
    return np.random.normal(scale=stddev, size=shape)

# Кількість класів у задачі класифікації
num_classes = 10

# Ініціалізація параметрів для кожного згорткового шару та повнозв'язаного шару згідно методу Ксав'є

# Перший згортковий шар
kernel_conv1 = initialize_weights((96, 3, 11, 11))  # 96 фільтрів розміром 11x11 з трьома входами кольору
bias_conv1 = np.zeros((kernel_conv1.shape[0], 1))  # Зсуви для кожного фільтра

# Другий згортковий шар
kernel_conv2 = initialize_weights((256, 96, 5, 5))  # 256 фільтрів розміром 5x5, кожен з 96 входів
bias_conv2 = np.zeros((kernel_conv2.shape[0], 1))  # Зсуви для кожного фільтра

# Третій згортковий шар
kernel_conv3 = initialize_weights((384, 256, 3, 3))  # 384 фільтри розміром 3x3, кожен з 256 входів
bias_conv3 = np.zeros((kernel_conv3.shape[0], 1))  # Зсуви для кожного фільтра

# Четвертий згортковий шар
kernel_conv4 = initialize_weights((384, 384, 3, 3))  # 384 фільтри розміром 3x3, кожен з 384 входів
bias_conv4 = np.zeros((kernel_conv4.shape[0], 1))  # Зсуви для кожного фільтра

# П'ятий згортковий шар
kernel_conv5 = initialize_weights((256, 384, 3, 3))  # 256 фільтрів розміром 3x3, кожен з 384 входів
bias_conv5 = np.zeros((kernel_conv5.shape[0], 1))  # Зсуви для кожного фільтра

# Повнозв'язаний шар 6
weights_f6 = initialize_weights((4096, 9216))  # Матриця ваг розміром 4096x9216
bias_f6 = np.zeros((weights_f6.shape[0], 1))  # Вектор зсуву

# Повнозв'язаний шар 7
weights_f7 = initialize_weights((4096, 4096))  # Матриця ваг розміром 4096x4096
bias_f7 = np.zeros((weights_f7.shape[0], 1))  # Вектор зсуву

# Вихідний повнозв'язаний шар
weights_output = initialize_weights((num_classes, 4096))  # Матриця ваг розміром 10x4096 (для 10 класів)
bias_output = np.zeros((weights_output.shape[0], 1))  # Вектор зсуву

# Створення списку параметрів для подальшого використання
params = [kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3,
          kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6,
          weights_f7, bias_f7, weights_output, bias_output]

In [ ]:
# Завантаження набору даних CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Перетворення типу даних на float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Нормалізація даних шляхом ділення на 255
X_train /= 255
X_test /= 255

# Перетворення міток на one-hot вектори
Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)

# Вибір підмножини даних для ефективнішого навчання

# Визначення розміру підмножини та обчислення кількості зразків для навчання
subset_fraction = 0.01
subset_size = int(len(X_train) * subset_fraction)

# Вибір випадкової підмножини з навчальних даних
subset_indices = np.random.choice(len(X_train), size=subset_size, replace=False)
X_train_subset = X_train[subset_indices]
Y_train_subset = Y_train[subset_indices]

# Вибір випадкової підмножини з тестових даних
subset_indices = np.random.choice(len(X_test), size=subset_size, replace=False)
X_test_subset = X_test[subset_indices]
Y_test_subset = Y_test[subset_indices]

# Зміна розміру зображень для моделі AlexNet

# Зміна розміру навчальних зображень
resized_images_train = []
for i in range(len(X_train_subset)):
    # Зміна розміру та перетворення порядку каналів зображення
    resized_images_train.append(np.transpose(cv2.resize(X_train_subset[i], (227, 227), interpolation=cv2.INTER_LINEAR), (2, 0, 1)))
X_train_subset_resized = np.array(resized_images_train)

# Зміна розміру тестових зображень
resized_images_test = []
for i in range(len(X_test_subset)):
    # Зміна розміру та перетворення порядку каналів зображення
    resized_images_test.append(np.transpose(cv2.resize(X_test_subset[i], (227, 227), interpolation=cv2.INTER_LINEAR), (2, 0, 1)))
X_test_subset_resized = np.array(resized_images_test)

In [ ]:
def train_model(epochs, batch_size, learning_rate, beta1, beta2, params):
    # Розпакування параметрів
    kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6, weights_f7, bias_f7, weights_output, bias_output = params

    # Отримання функцій прямого та зворотного поширення
    forward_propagation, backward_propagation = my_model()

    # Списки для збереження точності та втрат
    accuracy_list = []
    loss_list = []

    # Розділення навчальних даних на пакети
    batches = [X_train_subset_resized[i:i+batch_size] for i in range(0, len(X_train_subset_resized), batch_size)]

    # Початок великого циклу для кожної епохи
    total_start_time = time.time()
    for epoch in range(epochs):
        epoch_loss = 0
        start_time = time.time()

        # Цикл по пакетам
        for i in range(len(batches)):
            print(f"batch: {i}")
            # Ініціалізація градієнтів для кожного пакету
            d_kernel_conv1   = np.zeros(kernel_conv1.shape)
            d_bias_conv1     = np.zeros(bias_conv1.shape)
            d_kernel_conv2   = np.zeros(kernel_conv2.shape)
            d_bias_conv2     = np.zeros(bias_conv2.shape)
            d_kernel_conv3   = np.zeros(kernel_conv3.shape)
            d_bias_conv3     = np.zeros(bias_conv3.shape)
            d_kernel_conv4   = np.zeros(kernel_conv4.shape)
            d_bias_conv4     = np.zeros(bias_conv4.shape)
            d_kernel_conv5   = np.zeros(kernel_conv5.shape)
            d_bias_conv5     = np.zeros(bias_conv5.shape)
            d_weights_f6     = np.zeros(weights_f6.shape)
            d_bias_f6        = np.zeros(bias_f6.shape)
            d_weights_f7     = np.zeros(weights_f7.shape)
            d_bias_f7        = np.zeros(bias_f7.shape)
            d_weights_output = np.zeros(weights_output.shape)
            d_bias_output    = np.zeros(bias_output.shape)

            # Обчислення градієнтів за допомогою зворотного поширення для кожного зразка в пакеті
            for k in range(len(batches[i])):
                d_kernel_conv1_, d_bias_conv1_, d_kernel_conv2_, d_bias_conv2_, d_kernel_conv3_, d_bias_conv3_, d_kernel_conv4_, d_bias_conv4_, d_kernel_conv5_, d_bias_conv5_, d_weights_f6_, d_bias_f6_, d_weights_f7_, d_bias_f7_, d_weights_output_, d_bias_output_ = backward_propagation(batches[i][k], Y_train_subset[batch_size*i+k].reshape(-1, 1), learning_rate, params)
                d_kernel_conv1   += d_kernel_conv1_
                d_bias_conv1     += d_bias_conv1_
                d_kernel_conv2   += d_kernel_conv2_
                d_bias_conv2     += d_bias_conv2_
                d_kernel_conv3   += d_kernel_conv3_
                d_bias_conv3     += d_bias_conv3_
                d_kernel_conv4   += d_kernel_conv4_
                d_bias_conv4     += d_bias_conv4_
                d_kernel_conv5   += d_kernel_conv5_
                d_bias_conv5     += d_bias_conv5_
                d_weights_f6     += d_weights_f6_
                d_bias_f6        += d_bias_f6_
                d_weights_f7     += d_weights_f7_
                d_bias_f7        += d_bias_f7_
                d_weights_output += d_weights_output_
                d_bias_output    += d_bias_output_

            # Збереження градієнтів
            grads = [d_kernel_conv1, d_bias_conv1, d_kernel_conv2, d_bias_conv2, d_kernel_conv3, d_bias_conv3, d_kernel_conv4, d_bias_conv4, d_kernel_conv5, d_bias_conv5, d_weights_f6, d_bias_f6, d_weights_f7, d_bias_f7, d_weights_output, d_bias_output]
            params = [kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6, weights_f7, bias_f7, weights_output, bias_output]
            kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, kernel_conv4, bias_conv4, kernel_conv5, bias_conv5, weights_f6, bias_f6, weights_f7, bias_f7, weights_output, bias_output = adam(params, grads, learning_rate, len(batches[i]), beta1, beta2)
        # Обчислення часу, який зайняла епоха
        end_time = time.time()
        hours, remainder = divmod(end_time-start_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        # Оцінка точності та втрат для валідаційних даних
        predicted_labels = []
        for i in range(len(X_test_subset_resized)):
            image = X_test_subset_resized[i]
            label = Y_test_subset[i]
            output = forward_propagation(image, params)[11]
            loss = cross_entropy_loss(output, label)
            epoch_loss += loss
            predicted_label = np.argmax(output)
            predicted_labels.append(predicted_label)
        # Обчислення середньої втрати за епоху
        avg_epoch_loss = epoch_loss / len(X_test_subset_resized)
        # Додавання середньої втрати до списку втрат для подальшого аналізу
        loss_list.append(avg_epoch_loss)
        # Передбачення міток для валідаційних даних та оцінка точності
        predicted_labels = np.array(predicted_labels)
        true_labels = np.argmax(Y_test_subset, axis=1)
        accuracy = np.mean(predicted_labels == true_labels)
        # Додавання точності до списку точності для подальшого аналізу
        accuracy_list.append(accuracy)
        # Вивід інформації про поточну епоху, час її завершення, точність та середню втрату
        print(f"Epoch: {epoch+1}/{epochs} - Time: {int(hours)}h:{int(minutes)}m:{int(seconds)}s - Accuracy: {accuracy} - Loss: {avg_epoch_loss}")
    # Отримання часу завершення всього тренування
    total_end_time = time.time()
    # Розрахунок часу, який пройшов від початку до завершення тренування в годинах, хвилинах та секундах
    t_hours, t_remainder = divmod(total_end_time-total_start_time, 3600)
    t_minutes, t_seconds = divmod(t_remainder, 60)
    # Виведення загального часу тренування
    print(f"Total time: {int(t_hours)}h:{int(t_minutes)}m:{int(t_seconds)}s")

In [ ]:
epochs = 10  # Кількість епох тренування
batch_size = 16  # Розмір пакета для навчання
learning_rate = 0.0001  # Швидкість навчання
beta1 = 0.95  # Параметр бета1 для оптимізатора Adam
beta2 = 0.99  # Параметр бета2 для оптимізатора Adam

# Виклик функції тренування моделі зі встановленими параметрами
train_model(epochs, batch_size, learning_rate, beta1, beta2, params)